# Recourse Generation

The key point of this library - to implement recourse methods.

## Usage

Here is how to use a RecourseGenerator

In [2]:
# Import necessary components
import os
import sys
sys.path.append(os.path.join(os.path.dirname('rocelib'), '..'))

from rocelib.generators.recourse_methods.BinaryLinearSearch import BinaryLinearSearch
from rocelib.lib.models.pytorch_models.SimpleNNModel import SimpleNNModel
from rocelib.datasets.ExampleDatasets import get_example_dataset
from rocelib.lib.models.sklearn_models.LogisticRegressionModel import LogisticRegressionModel
from rocelib.lib.tasks.ClassificationTask import ClassificationTask
from rocelib.generators.robust_recourse_methods.STCE import TrexNN

# Load and preprocess dataset
dl = get_example_dataset("ionosphere")
dl.default_preprocess()

# Load model, note some RecourseGenerators may only work with a certain type of model,
# e.g., MCE only works with a SimpleNNModel
# model = LogisticRegressionModel()
model = SimpleNNModel(34, [10], 1)
# Create task
task = ClassificationTask(model, dl)

# Train model on dataset
task.train()

In [3]:
# Each RecourseGenerator takes the task on creation, it can also take a custom distance function
#recourse_gen = BinaryLinearSearch(task)
recourse_gen = TrexNN(task)
# Get negative instances, the default column_name is always "target" but you can set it to the
# name of your dataset's target variable
negs = dl.get_negative_instances(neg_value=0, column_name="target")

# You can generate for a set of instances stored in a DataFrame
print(recourse_gen.generate(negs.head(5)))

Restricted license - for non-production use only - expires 2025-11-24
   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
1   0.348433        0.0   0.721648  -0.527811   0.634308  -1.037587   
3   0.348433        0.0   0.721648  -1.125172   0.768477   1.921340   
5   0.348433        0.0  -1.243407  -0.114091  -1.349028  -0.511523   
7  -2.869990        0.0  -1.290430  -0.100661  -1.157858  -0.251849   
9   0.348433        0.0  -1.327935  -0.292560  -1.157858  -0.251849   

   feature_6  feature_7  feature_8  feature_9  ...  feature_24  feature_25  \
1  -1.339106  -2.029452   0.964074  -0.469482  ...   -1.037790   -0.383054   
3   0.329433  -2.152585  -1.010873  -0.375331  ...    1.045426    1.926340   
5  -1.133699  -0.456917  -0.720437  -0.237965  ...   -0.624980    0.109965   
7   0.914531  -2.152585  -1.010873  -0.375331  ...    1.045426    2.109592   
9  -1.118190  -0.229536  -0.784346  -0.930218  ...   -0.942070    0.114081   

   feature_26  feature_27  feature

In [4]:
# You can generate for one instance, that can be a Series
print(recourse_gen.generate_for_instance(negs.iloc[0]))

   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
1   0.348433        0.0   0.721648  -0.527811   0.634308  -1.037587   

   feature_6  feature_7  feature_8  feature_9  ...  feature_24  feature_25  \
1  -1.339106  -2.029452   0.964074  -0.469482  ...    -1.03779   -0.383054   

   feature_26  feature_27  feature_28  feature_29  feature_30  feature_31  \
1   -1.447849   -0.208419   -0.989185    -0.17353   -0.909063   -0.115213   

   feature_32  feature_33  
1   -0.932605   -0.083286  

[1 rows x 34 columns]


In [5]:
# You can generate for all negative instances
print(recourse_gen.generate_for_all(neg_value=0, column_name="target"))

     feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
1     0.348433        0.0   0.721648  -0.527811   0.634308  -1.037587   
3     0.348433        0.0   0.721648  -1.125172   0.768477   1.921340   
5     0.348433        0.0  -1.243407  -0.114091  -1.349028  -0.511523   
7    -2.869990        0.0  -1.290430  -0.100661  -1.157858  -0.251849   
9     0.348433        0.0  -1.327935  -0.292560  -1.157858  -0.251849   
..         ...        ...        ...        ...        ...        ...   
242   0.348433        0.0  -0.431111  -1.234948  -1.157858  -0.251849   
244   0.348433        0.0  -1.290430  -0.100661   0.768477   0.028254   
246   0.348433        0.0  -1.290430  -0.100661  -1.157858  -0.251849   
250   0.348433        0.0  -1.290430  -0.100661  -1.157858  -0.251849   
252   0.348433        0.0  -1.290430  -0.100661  -1.157858  -0.251849   

     feature_6  feature_7  feature_8  feature_9  ...  feature_24  feature_25  \
1    -1.339106  -2.029452   0.964074  -0.46

## Implementing your own RecourseGenerator

Here is an example of creating your own RecourseGenerator. Let's make a simple one which gets
n different positive instances and chooses a random one. Let's say it also allows a random seed value.

In [6]:
from rocelib.generators.RecourseGenerator import RecourseGenerator
import pandas as pd

# Implement the RecourseGenerator class
class RandomCE(RecourseGenerator):

    # You must implement the _generation_method function, this returns the CE for a given
    # instance, if you take any extra arguments make sure to specify them before **kwargs,
    # like we have done for n and seed (they must have some default value)
    def _generation_method(self, instance,
                           column_name="target", neg_value=0, n=50, seed=None, **kwargs):
        # Remember, the RecourseGenerator has access to its Task! Use this to get access to your dataset or model,
        # or to use any of their methods, here we use the ClassificationTask's get_random_positive_instance() method
        pos = pd.concat([self.task.get_random_positive_instance(neg_value=neg_value, column_name=column_name) for _ in range(n)])

        # Depending on whether a seed is provided, we return a random positive - the result must be a DataFrame
        if seed is None:
            return pos.sample(n=1)

        return pos.sample(n=1, random_state=seed)

Within the RecourseGenerator you can access:

- The Task - self.Task
- The DatasetLoader - self.task.training_data
- The BaseModel - self.task.model

and their respective methods. If your method needs additional arguments, you can put them in the function signature
but do NOT remove any other arguments (including **kwargs). Remember to return a DataFrame!

Here is our new CE in use below:

In [7]:
# Create RecourseGenerator
random_ce = RandomCE(task)

# Test it
ces = random_ce.generate_for_all()
print(ces)

     feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
1     0.348433        0.0   0.721648  -0.176998   0.768477  -0.241309   
3     0.348433        0.0   0.714505   0.046070   0.768477  -0.278709   
5     0.348433        0.0   0.721648  -0.250478   0.768477   1.115891   
7     0.348433        0.0   0.721648  -0.250478   0.768477   1.115891   
9     0.348433        0.0   0.721648  -0.008148   0.768477   0.008543   
..         ...        ...        ...        ...        ...        ...   
242   0.348433        0.0   0.544102  -0.300840   0.711823   0.131654   
244   0.348433        0.0   0.532674  -0.138251   0.732300  -0.295073   
246   0.348433        0.0   0.080761   0.134590   0.326133  -0.176722   
250   0.348433        0.0   0.389816   0.087585   0.262602  -0.571438   
252   0.348433        0.0   0.684747  -0.080834   0.731934  -0.334821   

     feature_6  feature_7  feature_8  feature_9  ...  feature_24  feature_25  \
1     0.914531  -0.461494   0.746139  -0.35

We can even verify it by seeing all the predictions for the CEs are positive.

In [8]:
print(model.predict(ces))

            0
0    0.674967
1    0.742654
2    0.698597
3    0.698597
4    0.724774
..        ...
120  0.734488
121  0.731971
122  0.601141
123  0.720799
124  0.736545

[125 rows x 1 columns]
